In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os
from datetime import date
import random
import pandas as pd


username = "admincol"
password = "password"
rows = "100"
timeout = 500
url = "https://prod.socoomni.com/smcfs/yfshttpdbi/sterlingdbqueryclient.jsp"
ruta_descarga = r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\Querys result\RPA\DescargaErrorMessage"
ruta_data = r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\Querys result"
mostrar = False



def iniciar(username, password, rows, timeout, ruta_descarga, url, mostrar ):
    dict_querys = init_querys()
    now=date.today()
    anio=now.year
    mes=now.month
    dia=now.day
    dia=dia-1
    fecha = "0" + str(mes) + " - " + "0" +str(dia)
    
    for key, value in dict_querys.items():
        print(key + '\n')
        query_name = key
        query_value = value
        descargar_archivosCiclo(username, password, query_value, rows, timeout, ruta_descarga, url, mostrar,fecha,anio,mes,dia)
        print(key + 'Finalizado' +'\n')

def descargar_archivosCiclo(username, password, query, rows, timeout, ruta_descarga, url, mostrar,fecha,anio,mes,dia)->None:
    options = Options() 
    options.headless = mostrar
    options.add_experimental_option('prefs',  {
        "download.default_directory": ruta_descarga,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
    })
    # Inicia sesión en Chrome
    driver = webdriver.Chrome(chrome_options=options)
    driver.implicitly_wait(30)

    # Acceder a la aplicación web
    driver.get(url)
    # Username
    search_username = driver.find_element("xpath","/html/body/div[2]/form[1]/div[1]/div/span/input")
    search_username.clear()
    search_username.send_keys(username)
    # Password
    search_pass = driver.find_element("xpath","/html/body/div[2]/form[1]/div[1]/div/span/span[1]/input")
    search_pass.clear()
    search_pass.send_keys(password)
    # Query
    search_query = driver.find_element("xpath","/html/body/div[2]/form[1]/table/tbody/tr[2]/td/textarea")
    search_query.clear()
    search_query.send_keys(query)
    # Rows
    search_rows = driver.find_element("xpath","/html/body/div[2]/form[1]/table/tbody/tr[3]/td/input[2]")
    search_rows.clear()
    search_rows.send_keys(rows)
    # RunSQL
    search_run = driver.find_element("xpath","/html/body/div[2]/form[1]/table/tbody/tr[3]/td/button")
    search_run.click()

    seriep= pd.Series(range(1,101))
    time.sleep(random.uniform(12.0,17.0))
    
    
    filas = len(driver.find_elements_by_xpath("/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr"))
    
    
    
    lista=[]
    for j in range(2,filas+1):
        data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(j)+"]/td[2]").text
        lista.append(data)
        
    seriep=pd.concat([seriep,pd.Series(lista)],axis=1)    
    

    
    lista=[]
    for k in range(2,filas+1):
        data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(k)+"]/td[9]").text
        lista.append(data)
    
    seriep=pd.concat([seriep,pd.Series(lista)],axis=1)
    
    seriep.columns=["A","EXPORT_KEY","MESSAGE"]
    seriep.to_csv(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\DescargaQueryMensajes\Error_Message" + str(anio) +str(mes)+ str(dia) + ".csv",sep=",",index=False)
    
    button1_name = driver.find_element(By.XPATH,"/html/body/div[2]/form[2]/div/div[2]/button[1]").get_attribute("name")
    
    if button1_name == "btnNext":
        seriep=seriep.drop(["EXPORT_KEY","MESSAGE"],axis=1)
        search_next = driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[2]/button[1]")
        search_next.click()
        time.sleep(random.uniform(12.0,17.0))
        filas = len(driver.find_elements_by_xpath("/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr"))
        
        lista=[]
        for j in range(2,filas+1):
            data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(j)+"]/td[2]").text
            lista.append(data)
        
        seriep=pd.concat([seriep,pd.Series(lista)],axis=1) 
        
        
        lista=[]
        for k in range(2,filas+1):
            data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(k)+"]/td[9]").text
            lista.append(data)
    
        seriep=pd.concat([seriep,pd.Series(lista)],axis=1)
        
        seriep.columns=["A","EXPORT_KEY","MESSAGE"]
        seriep.to_csv(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\DescargaQueryMensajes\Error_Message"+ str(anio)  + str(mes)+ str(dia) + ".csv",mode='a',header=False,index=False)
        
        button1_name = driver.find_element(By.XPATH,"/html/body/div[2]/form[2]/div/div[2]/button[1]").get_attribute("name")
        button2_name = driver.find_element(By.XPATH,"/html/body/div[2]/form[2]/div/div[2]/button[2]").get_attribute("name")
        
        
        while button2_name == "btnNext": 
            seriep=seriep.drop(["EXPORT_KEY","MESSAGE"],axis=1)
            search_next = driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[2]/button[2]")
            search_next.click()
            time.sleep(random.uniform(12.0,17.0))
            filas = len(driver.find_elements_by_xpath("/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr"))
        
            lista=[]
            for j in range(2,filas+1):
                data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(j)+"]/td[2]").text
                lista.append(data)
        
            seriep=pd.concat([seriep,pd.Series(lista)],axis=1) 
        
            lista=[]
            for k in range(2,filas+1):
                data=driver.find_element("xpath","/html/body/div[2]/form[2]/div/div[3]/table/tbody/tr["+str(k)+"]/td[9]").text
                lista.append(data)
    
            seriep=pd.concat([seriep,pd.Series(lista)],axis=1)
        
            seriep.columns=["A","EXPORT_KEY","MESSAGE"]
            seriep.to_csv(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\DescargaQueryMensajes\Error_Message\Error_Message"+ str(anio) + str(mes)+ str(dia) + ".csv",mode='a',header=False,index=False)
            
        
            button2_name = driver.find_element(By.XPATH,"/html/body/div[2]/form[2]/div/div[2]/button[2]").get_attribute("name")
        
        driver.quit()
    
    else: 
        driver.quit()
    
def init_querys()->dict:
    query_name_list = ['9) ERROR MESSAGE']
    query_code_list = []
    for i in range(len(query_name_list)):
        ruta_archivo = 'C:/Users/luis.montoya/OneDrive - Grupo Socofar/Dashboard OMS/Querys result/RPA/' + query_name_list[i] + '.txt'
        query = abrir_query(ruta_archivo)
        query_code_list.append(query)

    #Almaceno los nombres y consultas en un diccionario
    querys_dict = dict(zip(query_name_list,query_code_list))
    return querys_dict

def abrir_query(ruta_archivo)->str:
    archivo = open(ruta_archivo,'r')
    query = archivo.read()
    archivo.close()
    return query


username = "admincol"
password = "password"
rows = "100"
timeout = 500
url = "https://prod.socoomni.com/smcfs/yfshttpdbi/sterlingdbqueryclient.jsp"
ruta_descarga = r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\Querys result\RPA\DescargaErrorMessage"
ruta_data = r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\Dashboard OMS\Querys result"
mostrar = False




iniciar(username, password, rows, timeout, ruta_descarga, url, mostrar)

print("Proceso finalizado")

    

9) ERROR MESSAGE



C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_11280\641858376.py:51: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


AttributeError: 'WebDriver' object has no attribute 'find_elements_by_xpath'

In [5]:
import re
from datetime import date

def recorta(linea):
    linea_nueva= linea[linea.find(",")+1:linea.find(",")+12] + "," + linea[linea.find(",",8)+1:len(linea)]
    return linea_nueva


current_time=date.today()
anio=current_time.year
mes=current_time.month
dia=current_time.day
dia=dia-1

df=pd.read_csv(r"C:\Users\luis.montoya\Grupo Socofar\Nicolas Chavarro Santana - Dashboard OMS\DescargaQueryMensajes\Error_Message" + str(anio) + str (mes) + str (dia) + ".csv",sep=',')
data=df["A"]
df_salida_total =pd.DataFrame(columns=["Consecutivo","Sku","Cantidad","Fecha","Franja","Suc","Mensaje","Mensaje2"])



llave=[]
exportkey_full=[]
llave_extraccion=1

for valorcadena in data:
    extraccion_exportkey= valorcadena[valorcadena.find(",")+1:valorcadena.find(",")+24]
    llave.append(llave_extraccion)
    exportkey_full.append(extraccion_exportkey)
    llave_extraccion=llave_extraccion+1
    

base_cruce=pd.concat([pd.Series(llave),pd.Series(exportkey_full)],axis=1)
base_cruce.columns=["Llave","Exportkey"]

data = data.apply(recorta)

data_aux=pd.concat([pd.Series(llave),pd.Series(data)],axis=1)
data_aux.columns=["Llave","Mensaje"]

data_aux=data_aux.drop_duplicates(data_aux.columns[~data_aux.columns.isin(["Llave"])],keep='first')

data=data_aux["Mensaje"]

a=0

for celda in data:
    cadenatotal=celda
    patron= r'<Message Text=(.*?)/>'
    resultado=re.findall(patron,cadenatotal)
    codsuc= []
    consecutivo = []
    for l in resultado:
        inicio =l.find("-")+2
        fin = l.find('"',inicio,5)-1
        var = l[inicio:fin]
        codsuc.append(var)
        consecutivo.append(data_aux.iloc[a,0])
    
    fecha = []
    franja=[]
    fechatotal = cadenatotal[0:4] + "/" + cadenatotal[4:6] + "/" + cadenatotal[6:8]
    franjatotal= cadenatotal[8:11]
    
    for i in range(0,len(codsuc)):
        fecha.append(fechatotal)
        franja.append(franjatotal)
    
    aux=0
    for i in resultado:
        corte= i.find("-")-1
        resultado[aux]=i[2:corte]
        aux=aux+1
    resultado
    
    patron = r'ItemGroupCode(.*?)LineId'
    skus=re.findall(patron,cadenatotal)
    aux=0
    for i in skus:
        corte=i.find("ID")+5
        fin_corte=i.find('"',corte,len(i)+1)
        skus[aux]=i[corte:fin_corte]
        aux=aux+1
    skus
    
    
    patron=r'UnavailableReason=""(.*?)"'
    mensajedef=re.findall(patron,cadenatotal)
    patron = r'<Messages>(.*?)</Messages>'
    comparacion=re.findall(patron,cadenatotal)
    
    patron=r'<Message Text=(.*?)/>'
    patron_cantidad= r'RequiredQty=""(.*?)"'
    cantidades_skus=re.findall(patron_cantidad,cadenatotal)
    cantidad=[]
    skuId=[]
    mensajeb=[]
    aux=0
    print("Inicia un nuevo conjunto de skus" + "(" +str(a) + ")")
    for i in comparacion:
        elementos=re.findall(patron,i)
        print(skus[aux])
        print(len(elementos))
        for j in range(1,len(elementos)+1):
            skuId.append(skus[aux])
            mensajeb.append(mensajedef[aux])
            cantidad.append(cantidades_skus[aux])
        aux=aux+1
    
    
    df_salida=pd.concat([pd.Series(consecutivo),pd.Series(skuId),pd.Series(cantidad),pd.Series(fecha),pd.Series(franja),pd.Series(codsuc),pd.Series(resultado),pd.Series(mensajeb)],axis=1)
    df_salida.columns=["Consecutivo","Sku","Cantidad","Fecha","Franja","Suc","Mensaje","Mensaje2"]
    
    df_salida_total = pd.concat([df_salida_total,df_salida],axis=0)
    df_salida.drop(["Consecutivo","Sku","Cantidad","Fecha","Franja","Suc","Mensaje","Mensaje2"],axis=1)
    a=a+1
    
sucursales_oms= ["COCV_654","COCV_688","COCV_666","COCV_739","COCV_643","COCV_468","COCV_307","COCV_171","COCV_268","COCV_734","COCV_411","COCV_392","COCV_562","COCV_156","COCV_545","COCV_551","COCV_549","COCV_577","COCV_513","COCV_677","COCV_381","COCV_388","COCV_189","COCV_782","COCV_578","COCV_816","COCV_229","COCV_554","COCV_98","COCV_181","COCV_378","COCV_573","COCV_101","COCV_556","COCV_390","COCV_183","COCV_155","COCV_190","COCV_297","COCV_401","COCV_262","COCV_402","COCV_394","COCV_162","COCV_174","COCV_135","COCV_447","COCV_508","COCV_250","COCV_452","COCV_696","COCV_471","COCV_630","COCV_453","COCV_391","COCV_522","COCV_294","COCV_606","COCV_792","COCV_260","COCV_117","COCV_122","COCV_133","COCV_185","COCV_257","COCV_266","COCV_293","COCV_596","COCV_679","COCV_215","COCV_831","COCV_832","COCV_470","COCV_559","COCV_843","COCV_144","COCV_457","COCV_131","COCV_249","COCV_201","COCV_830","COCV_119","COCV_460","COCV_424","COCV_425","COCV_264","COCV_756","COCV_428","COCV_498","COCV_885","COCV_881","COCV_873"]

df_salida_total=df_salida_total[df_salida_total.Suc.isin(sucursales_oms)]
df_salida_total=df_salida_total.drop_duplicates()

df_salida_total=pd.merge(df_salida_total,base_cruce,left_on="Consecutivo",right_on="Llave", how="left")

df_salida_total=df_salida_total.drop_duplicates(df_salida_total.columns[~df_salida_total.columns.isin(["Consecutivo","Cantidad","Llave","Exportkey"])])
df_salida_total=df_salida_total.drop(["Consecutivo","Llave"],axis=1)

df_salida_total.to_csv(r"C:\Users\luis.montoya\Grupo Socofar\Nicolas Chavarro Santana - Dashboard OMS\DescargaQueryMensajes\ArchivosDepurados\MensajesDepurado" + str(anio) + str(mes) + str(dia) + ".csv",sep=",",index=False)
print("Proceso Finalizado")
   

Inicia un nuevo conjunto de skus(0)
20682
6
Inicia un nuevo conjunto de skus(1)
102189
3
Inicia un nuevo conjunto de skus(2)
392556
3
Inicia un nuevo conjunto de skus(3)
54358
2
Inicia un nuevo conjunto de skus(4)
18650
3
146775
3
Inicia un nuevo conjunto de skus(5)
86440
6
Inicia un nuevo conjunto de skus(6)
54358
3
Inicia un nuevo conjunto de skus(7)
116515
33
Inicia un nuevo conjunto de skus(8)
146775
3
Inicia un nuevo conjunto de skus(9)
116515
33
Inicia un nuevo conjunto de skus(10)
392556
2
Inicia un nuevo conjunto de skus(11)
399539
1
Inicia un nuevo conjunto de skus(12)
270964
2
Inicia un nuevo conjunto de skus(13)
45002
5
Inicia un nuevo conjunto de skus(14)
45002
2
Inicia un nuevo conjunto de skus(15)
289390
2
Inicia un nuevo conjunto de skus(16)
86440
6
Inicia un nuevo conjunto de skus(17)
21600
5
Inicia un nuevo conjunto de skus(18)
21600
5
Inicia un nuevo conjunto de skus(19)
56143
6
Inicia un nuevo conjunto de skus(20)
264348
2
Inicia un nuevo conjunto de skus(21)
20725
3

Inicia un nuevo conjunto de skus(200)
297563
3
Inicia un nuevo conjunto de skus(201)
297563
5
Inicia un nuevo conjunto de skus(202)
268911
4
Inicia un nuevo conjunto de skus(203)
258978
2
Inicia un nuevo conjunto de skus(204)
97951
6
Inicia un nuevo conjunto de skus(205)
291936
3
Inicia un nuevo conjunto de skus(206)
291936
2
Inicia un nuevo conjunto de skus(207)
291936
1
Inicia un nuevo conjunto de skus(208)
392323
4
Inicia un nuevo conjunto de skus(209)
266034
1
272657
1
Inicia un nuevo conjunto de skus(210)
294658
5
294652
5
Inicia un nuevo conjunto de skus(211)
391177
5
Inicia un nuevo conjunto de skus(212)
261571
4
259258
4
Inicia un nuevo conjunto de skus(213)
392690
4
Inicia un nuevo conjunto de skus(214)
392690
4
Inicia un nuevo conjunto de skus(215)
391177
5
Inicia un nuevo conjunto de skus(216)
21426
6
Inicia un nuevo conjunto de skus(217)
64282
2
Inicia un nuevo conjunto de skus(218)
292713
1
266035
1
293468
1
298787
1
293150
1
277429
1
274268
1
Inicia un nuevo conjunto de s

Inicia un nuevo conjunto de skus(367)
21162
5
Inicia un nuevo conjunto de skus(368)
274648
1
Inicia un nuevo conjunto de skus(369)
292601
2
94352
2
256767
2
Inicia un nuevo conjunto de skus(370)
292601
2
94352
2
256767
2
Inicia un nuevo conjunto de skus(371)
19948
6
Inicia un nuevo conjunto de skus(372)
399539
2
Inicia un nuevo conjunto de skus(373)
386257
5
Inicia un nuevo conjunto de skus(374)
386257
5
Inicia un nuevo conjunto de skus(375)
386257
8
Inicia un nuevo conjunto de skus(376)
104030
2
1159
2
32237
2
Inicia un nuevo conjunto de skus(377)
296683
3
Inicia un nuevo conjunto de skus(378)
386257
1
255042
1
Inicia un nuevo conjunto de skus(379)
295262
2
392665
2
Inicia un nuevo conjunto de skus(380)
289605
2
Inicia un nuevo conjunto de skus(381)
275360
1
Inicia un nuevo conjunto de skus(382)
386257
2
255042
2
Inicia un nuevo conjunto de skus(383)
19948
2
Inicia un nuevo conjunto de skus(384)
386257
4
Inicia un nuevo conjunto de skus(385)
256500
1
Inicia un nuevo conjunto de skus(3

Inicia un nuevo conjunto de skus(529)
120109
34
Inicia un nuevo conjunto de skus(530)
255987
2
Inicia un nuevo conjunto de skus(531)
138626
2
167215
2
131836
2
125223
2
Inicia un nuevo conjunto de skus(532)
131836
6
Inicia un nuevo conjunto de skus(533)
393393
2
Inicia un nuevo conjunto de skus(534)
288411
5
Inicia un nuevo conjunto de skus(535)
288411
2
Inicia un nuevo conjunto de skus(536)
126440
6
Inicia un nuevo conjunto de skus(537)
95272
4
Inicia un nuevo conjunto de skus(538)
261844
2
Inicia un nuevo conjunto de skus(539)
18650
6
Inicia un nuevo conjunto de skus(540)
296432
2
264633
2
72140
2
Inicia un nuevo conjunto de skus(541)
259435
2
Inicia un nuevo conjunto de skus(542)
126440
34
Inicia un nuevo conjunto de skus(543)
95418
5
Inicia un nuevo conjunto de skus(544)
133771
2
Inicia un nuevo conjunto de skus(545)
297252
1
Inicia un nuevo conjunto de skus(546)
295670
2
Inicia un nuevo conjunto de skus(547)
295670
2
Inicia un nuevo conjunto de skus(548)
288411
4
Inicia un nuevo c

Inicia un nuevo conjunto de skus(682)
137560
34
161576
34
Inicia un nuevo conjunto de skus(683)
95725
31
Inicia un nuevo conjunto de skus(684)
95725
2
Inicia un nuevo conjunto de skus(685)
105171
6
Inicia un nuevo conjunto de skus(686)
86678
2
159030
2
Inicia un nuevo conjunto de skus(687)
296156
3
Inicia un nuevo conjunto de skus(688)
199271
3
Inicia un nuevo conjunto de skus(689)
99373
3
18650
3
Inicia un nuevo conjunto de skus(690)
72912
5
Inicia un nuevo conjunto de skus(691)
18650
3
Inicia un nuevo conjunto de skus(692)
72912
5
Inicia un nuevo conjunto de skus(693)
72912
5
Inicia un nuevo conjunto de skus(694)
72912
5
Inicia un nuevo conjunto de skus(695)
97951
6
Inicia un nuevo conjunto de skus(696)
158023
1
Inicia un nuevo conjunto de skus(697)
97982
2
21114
2
159535
2
Inicia un nuevo conjunto de skus(698)
21114
6
Inicia un nuevo conjunto de skus(699)
256500
1
Inicia un nuevo conjunto de skus(700)
158023
2
Inicia un nuevo conjunto de skus(701)
55358
3
Inicia un nuevo conjunto de

...

Inicia un nuevo conjunto de skus(1392)
399454
4
Inicia un nuevo conjunto de skus(1393)
20709
3
Inicia un nuevo conjunto de skus(1394)
138853
5
Inicia un nuevo conjunto de skus(1395)
138853
5
Inicia un nuevo conjunto de skus(1396)
300644
1
Inicia un nuevo conjunto de skus(1397)
161381
6
Inicia un nuevo conjunto de skus(1398)
113412
35
Inicia un nuevo conjunto de skus(1399)
114066
2
137779
2
Inicia un nuevo conjunto de skus(1400)
97951
6
Inicia un nuevo conjunto de skus(1401)
21426
5
Inicia un nuevo conjunto de skus(1402)
101697
2
Inicia un nuevo conjunto de skus(1403)
295633
1
295632
1
Inicia un nuevo conjunto de skus(1404)
286973
2
294446
2
Inicia un nuevo conjunto de skus(1405)
387009
33
Inicia un nuevo conjunto de skus(1406)
286973
2
Inicia un nuevo conjunto de skus(1407)
30390
1
Inicia un nuevo conjunto de skus(1408)
96556
2
Inicia un nuevo conjunto de skus(1409)
40445
30
Inicia un nuevo conjunto de skus(1410)
386403
2
Inicia un nuevo conjunto de skus(1411)
147585
2
Inicia un nuevo 

Inicia un nuevo conjunto de skus(1577)
167215
5
Inicia un nuevo conjunto de skus(1578)
146775
2
Inicia un nuevo conjunto de skus(1579)
146775
5
Inicia un nuevo conjunto de skus(1580)
71622
2
Inicia un nuevo conjunto de skus(1581)
129080
33
Inicia un nuevo conjunto de skus(1582)
147271
2
Inicia un nuevo conjunto de skus(1583)
272971
6
Inicia un nuevo conjunto de skus(1584)
129080
33
Inicia un nuevo conjunto de skus(1585)
99375
5
Inicia un nuevo conjunto de skus(1586)
105171
4
Inicia un nuevo conjunto de skus(1587)
159537
6
Inicia un nuevo conjunto de skus(1588)
71622
2
Inicia un nuevo conjunto de skus(1589)
257630
2
Inicia un nuevo conjunto de skus(1590)
274698
1
Inicia un nuevo conjunto de skus(1591)
167215
5
21426
5
Inicia un nuevo conjunto de skus(1592)
300628
2
Inicia un nuevo conjunto de skus(1593)
99375
5
Inicia un nuevo conjunto de skus(1594)
274698
3
Inicia un nuevo conjunto de skus(1595)
64414
2
Inicia un nuevo conjunto de skus(1596)
96556
35
Inicia un nuevo conjunto de skus(15

150925
2
Inicia un nuevo conjunto de skus(1752)
105171
4
Inicia un nuevo conjunto de skus(1753)
269471
4
Inicia un nuevo conjunto de skus(1754)
269471
3
Inicia un nuevo conjunto de skus(1755)
269471
3
Inicia un nuevo conjunto de skus(1756)
56143
6
Inicia un nuevo conjunto de skus(1757)
19276
7
Inicia un nuevo conjunto de skus(1758)
276516
2
Inicia un nuevo conjunto de skus(1759)
294831
2
Inicia un nuevo conjunto de skus(1760)
392049
2
Inicia un nuevo conjunto de skus(1761)
58066
1
Inicia un nuevo conjunto de skus(1762)
56143
6
97951
6
Inicia un nuevo conjunto de skus(1763)
399988
3
392049
3
Inicia un nuevo conjunto de skus(1764)
58066
1
Inicia un nuevo conjunto de skus(1765)
161443
4
Inicia un nuevo conjunto de skus(1766)
20725
3
Inicia un nuevo conjunto de skus(1767)
64074
29
99374
29
30920
29
168047
29
Inicia un nuevo conjunto de skus(1768)
109496
5
Inicia un nuevo conjunto de skus(1769)
120908
6
Inicia un nuevo conjunto de skus(1770)
104628
34
Inicia un nuevo conjunto de skus(1771)


Inicia un nuevo conjunto de skus(1916)
120109
34
Inicia un nuevo conjunto de skus(1917)
80479
3
Inicia un nuevo conjunto de skus(1918)
21426
6
388920
6
Inicia un nuevo conjunto de skus(1919)
98202
4
Inicia un nuevo conjunto de skus(1920)
298836
2
Inicia un nuevo conjunto de skus(1921)
168354
30
132433
30
105733
30
Inicia un nuevo conjunto de skus(1922)
51766
33
18034
33
123157
33
129080
33
Inicia un nuevo conjunto de skus(1923)
82516
2
Inicia un nuevo conjunto de skus(1924)
107152
29
Inicia un nuevo conjunto de skus(1925)
19611
29
Inicia un nuevo conjunto de skus(1926)
156955
6
Inicia un nuevo conjunto de skus(1927)
294318
1
Inicia un nuevo conjunto de skus(1928)
261675
2
Inicia un nuevo conjunto de skus(1929)
110600
2
Inicia un nuevo conjunto de skus(1930)
110600
3
Inicia un nuevo conjunto de skus(1931)
388920
3
75239
2
126099
3
Inicia un nuevo conjunto de skus(1932)
61666
6
Inicia un nuevo conjunto de skus(1933)
38039
31
Inicia un nuevo conjunto de skus(1934)
138239
4
Inicia un nuevo

Inicia un nuevo conjunto de skus(2089)
264348
2
Inicia un nuevo conjunto de skus(2090)
262715
2
Inicia un nuevo conjunto de skus(2091)
96556
31
Inicia un nuevo conjunto de skus(2092)
55505
3
Inicia un nuevo conjunto de skus(2093)
76107
31
Inicia un nuevo conjunto de skus(2094)
263580
1
Inicia un nuevo conjunto de skus(2095)
113412
31
Inicia un nuevo conjunto de skus(2096)
18702
3
Inicia un nuevo conjunto de skus(2097)
268759
2
Inicia un nuevo conjunto de skus(2098)
119410
3
18034
2
Inicia un nuevo conjunto de skus(2099)
387559
29
Inicia un nuevo conjunto de skus(2100)
287831
1
Inicia un nuevo conjunto de skus(2101)
111534
6
Inicia un nuevo conjunto de skus(2102)
285994
3
Inicia un nuevo conjunto de skus(2103)
21504
2
Inicia un nuevo conjunto de skus(2104)
167936
2
18034
2
Inicia un nuevo conjunto de skus(2105)
18034
4
Inicia un nuevo conjunto de skus(2106)
42596
2
Inicia un nuevo conjunto de skus(2107)
162462
3
Inicia un nuevo conjunto de skus(2108)
162462
3
Inicia un nuevo conjunto de